In [121]:
import numpy as np
import pandas as pd
import scipy.io as sio
from Bio import SeqIO
import collections

import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
import random

def load_test_sets(filename):
    go_data = sio.loadmat(filename, squeeze_me=True)
    go_terms = go_data['goTerm_labels'] # names of gene ontology function terms
    train_annotations = np.asarray(go_data['trainProts_label'].todense()) # training set of function annotations
    valid_annotations = np.asarray(go_data['validProts_label'].todense()) # valid "" ""
    test_annotations = np.asarray(go_data['testProts_label'].todense()) # test "" ""
    train_inds = go_data['trainProts']
    train_inds = train_inds - 1
    valid_inds = go_data['validProts']
    valid_inds = valid_inds - 1
    test_inds = go_data['testProts']
    test_inds = test_inds - 1 # subtract 1 for matlab index conversion into python

    return train_inds, valid_inds, test_inds, train_annotations, valid_annotations, test_annotations, go_terms

def load_FASTA(filename):
    """ Loads fasta file and returns a list of the Bio SeqIO records """
    infile = open(filename)
    full_entries = list(SeqIO.parse(infile, 'fasta'))
    sequences = [str(entry.seq) for entry in full_entries]
    names = [str(entry.id) for entry in full_entries]

    return sequences, names

#### Load human sequences and create train/dev/test sets, create lengths for FastText

In [122]:
human_sequences, human_protein_names = load_FASTA('../data/human_sequences.fasta')
human_train_idx, human_valid_idx, human_test_idx, human_train_labels, human_valid_labels, \
    human_test_labels, human_GO_terms = load_test_sets('../data/human_annotations_temporal_holdout.mat')

# Create train, validation, and test sets from the full list of human proteins
human_train_sequences = [human_sequences[i] for i in human_train_idx]
human_valid_sequences = [human_sequences[i] for i in human_valid_idx]
human_test_sequences = [human_sequences[i] for i in human_test_idx]

# Convert corresponding labels for train, validation, and test sets from the full list of human proteins.
human_train_labels = torch.from_numpy(human_train_labels).type(torch.LongTensor)
human_valid_labels = torch.from_numpy(human_valid_labels).type(torch.LongTensor)
human_test_labels = torch.from_numpy(human_test_labels).type(torch.LongTensor)

# Create lengths for sequence representation averaging in FastText
human_train_lengths = torch.LongTensor([len(human_train_sequences[i]) for i in range(len(human_train_sequences))])
human_valid_lengths = torch.LongTensor([len(human_valid_sequences[i]) for i in range(len(human_valid_sequences))])
human_test_lengths = torch.LongTensor([len(human_test_sequences[i]) for i in range(len(human_test_sequences))])

#### Load yeast sequences and create train/dev/test sets, create lengths for FastText

In [124]:
# Load yeast sequences and training data
yeast_sequences, yeast_protein_names = load_FASTA('../data/yeast_sequences.fasta')
yeast_train_idx, yeast_valid_idx, yeast_test_idx, yeast_train_labels, yeast_valid_labels, \
    yeast_test_labels, yeast_GO_terms = load_test_sets('../data/yeast_MF_temporal_holdout.mat')

# Create train, validation, and test sets from the full list of yeast proteins
yeast_train_sequences = [yeast_sequences[i] for i in yeast_train_idx]
yeast_valid_sequences = [yeast_sequences[i] for i in yeast_valid_idx]
yeast_test_sequences = [yeast_sequences[i] for i in yeast_test_idx]

# Convert corresponding labels for train, validation, and test sets from the full list of yeast proteins.
yeast_train_labels = torch.from_numpy(yeast_train_labels).type(torch.LongTensor)
yeast_valid_labels = torch.from_numpy(yeast_valid_labels).type(torch.LongTensor)
yeast_test_labels = torch.from_numpy(yeast_test_labels).type(torch.LongTensor)

# Create lengths for sequence representation averaging in FastText
yeast_train_lengths = torch.LongTensor([len(yeast_train_sequences[i]) for i in range(len(yeast_train_sequences))])
yeast_valid_lengths = torch.LongTensor([len(yeast_valid_sequences[i]) for i in range(len(yeast_valid_sequences))])
yeast_test_lengths = torch.LongTensor([len(yeast_test_sequences[i]) for i in range(len(yeast_test_sequences))])

#### Embed amino-acid chains

In [125]:
# Each amino-acid string becomes an NxD entry in a tensor, where N is the number of 
# amino-acid strings and D is the length of the longest chain in the set. 

ConvertCharToInt = {'A':1, 'B':2, 'C':3, 'D':4, 'E':5, 'F':6, 'G':7, 'H':8, 'I':9, 'J':10,
                   'K':11, 'L':12, 'M':13, 'N':14, 'O':15, 'P':16, 'Q':17, 'R':18, 'S':19,
                   'T':20, 'U':21, 'V':22, 'W':23, 'X':24, 'Y':25, 'Z':26}

def vectorize_AAs(string):
    '''This function takes an amino-acid string as input and outputs a vector of integers, with each
    integer representing one amino acid.
    
    For example, 'BACEA' is converted to [2, 1, 3, 5, 1]
    '''
    character_list = list(string) #converts 'BACEA' to ['B','A','C','E','A]
    for i in range(len(character_list)):
        character_list[i] = ConvertCharToInt[character_list[i]] #convert the character to a number
    return character_list

def AddZeros(vector, max_length):
    '''This function adds the necessary number of zeros and returns an array'''
    #max_length = length of longest vector in the batch
    #oldvector = initial vector for that amino-acid chain (in integers)
    diff = max_length - len(vector)
    if diff>0:
        ZerosToAdd = np.zeros(diff)
        vector.extend(ZerosToAdd)
    return vector 

def TransformAAsToTensor(ListOfSequences):
    '''This function takes as input a list of amino acid strings and creates a tensor matrix
    of dimension NxD, where N is the number of strings and D is the length of the longest AA chain
    
    "ListOfSequences" can be training, validation, or test sets
    '''
    #find longest amino-acid sequence
    max_length = len(max(ListOfSequences, key=len))
    Sequences = ListOfSequences.copy() 
    for AA in range(len(Sequences)): #for each amino-acid sequence
        Sequences[AA] = vectorize_AAs(Sequences[AA])
        Sequences[AA] = AddZeros(Sequences[AA], max_length)
    NewTensor = torch.from_numpy(np.array(Sequences))
    return NewTensor

In [126]:
human_train_vectors = TransformAAsToTensor(human_train_sequences)
human_valid_vectors = TransformAAsToTensor(human_valid_sequences)
human_test_vectors = TransformAAsToTensor(human_test_sequences)

In [127]:
yeast_train_vectors = TransformAAsToTensor(yeast_train_sequences)
yeast_valid_vectors = TransformAAsToTensor(yeast_valid_sequences)
yeast_test_vectors = TransformAAsToTensor(yeast_test_sequences)

## Train Model

### 0) Hyperparameter setting

### 1) Get batch data method:

In [128]:
import random

def batch_iter(TrainSeqs, yTrain, TrainSeqsLength, batch_size):
    start = -1 * batch_size
    dataset_size = TrainSeqs.size()[0]
    order = list(range(dataset_size))
    random.shuffle(order)

    while True:
        start += batch_size
        if start > dataset_size - batch_size:
            # Start another epoch.
            start = 0
            random.shuffle(order)
        batch_indices = order[start:start + batch_size]
        batch_indices_tensor = torch.LongTensor(batch_indices)
        batch_train = TrainSeqs[batch_indices_tensor].type(torch.LongTensor)
        batch_train_labels = yTrain[batch_indices_tensor]
        length_batch = TrainSeqsLength[batch_indices_tensor]
        yield [Variable(batch_train), Variable(batch_train_labels), Variable(length_batch)]  

### 2) FastText class:

In [129]:
class FastText(nn.Module):
    """
    FastText model
    """
       
    def __init__(self, vocab_size, emb_dim, num_labels):
        """
        @param vocab_size: size of the vocabulary. 
        @param emb_dim: size of the word embedding
        """
        super(FastText, self).__init__()

        self.num_labels = num_labels
        self.embed = nn.Embedding(vocab_size+1, emb_dim, padding_idx=0)
        self.linear = nn.Linear(emb_dim,num_labels)
        self.init_weights()
    
    def forward(self, data, length):
        """
        @param data: matrix of size (batch_size, max_sentence_length). Each row in data represents a 
            review that is represented using n-gram index. Note that they are padded to have same length.
        @param length: an int tensor of size (batch_size), which represents the non-trivial (excludes padding)
            length of each sentences in the data.
        """
        data = data.type(torch.LongTensor)
        out = self.embed(data)
        out = torch.sum(out, dim=1)
        out = out / length.view(-1,1).float()
            
        out = self.linear(out)
        return nn.functional.sigmoid(out)
    
    def init_weights(self):
        initrange = 0.1
        lin_layers = [self.linear]
        em_layer = [self.embed]
     
        for layer in lin_layers+em_layer:
            layer.weight.data.uniform_(-initrange, initrange)
            if layer in lin_layers:
                layer.bias.data.fill_(0)
                

### 2B) LSTM class:  

In [130]:
class LSTM(nn.Module):
    """
    LSTM model
    """  
    def __init__(self, vocab_size, emb_dim, hidden_size, num_labels, batch_size):
        """
        @param vocab_size: size of the vocabulary. 
        @param emb_dim: size of the word embedding
        """
        super(LSTM, self).__init__()
        self.num_labels = num_labels
        self.emb_dim = emb_dim
        self.vocab_size = vocab_size
        self.hidden_size = hidden_size
        self.batch_size = batch_size

        self.embed = nn.Embedding(vocab_size, emb_dim, padding_idx=0)
        self.linear_f = nn.Linear(emb_dim + hidden_size, hidden_size)
        self.linear_i = nn.Linear(emb_dim + hidden_size, hidden_size)
        self.linear_ctilde = nn.Linear(emb_dim + hidden_size, hidden_size)
        self.linear_o = nn.Linear(emb_dim + hidden_size, hidden_size)
    
        self.decoder = nn.Linear(hidden_size, num_labels)
        self.init_weights()
    
    def forward(self, data, hidden, c):
        """
        @param data: matrix of size (batch_size, max_sentence_length). Each row in data represents a 
            review that is represented using n-gram index. Note that they are padded to have same length.
        @param length: an int tensor of size (batch_size), which represents the non-trivial (excludes padding)
            length of each sentences in the data.
        """
        data = data.type(torch.LongTensor)
        emb = self.embed(data)
        embs = torch.chunk(emb, emb.size()[1], 1)
        
        def step(emb, hid, c_t):
            combined = torch.cat((hid,emb),1)
            f = F.sigmoid(self.linear_f(combined))
            i = F.sigmoid(self.linear_i(combined))
            c_tilde = F.tanh(self.linear_ctilde(combined))
            c_t = f*c_t + i*c_tilde
            o = F.sigmoid(self.linear_o(combined))
            hid = o * F.tanh(c_t)
            return hid, c_t
        
        for i in range(len(embs)):
            hidden, c = step(embs[i].squeeze(), hidden, c)
        output = self.decoder(hidden)
        return output, hidden
    
    def init_hidden(self):
        h0 = Variable(torch.zeros(self.batch_size, self.hidden_size))
        c0 = Variable(torch.zeros(self.batch_size, self.hidden_size))
        return h0, c0
        
    def init_weights(self):
        initrange = 0.1
        lin_layers = [self.linear_f, self.linear_i, self.linear_ctilde, self.linear_o]
        em_layer = [self.embed]
     
        for layer in lin_layers+em_layer:
            layer.weight.data.uniform_(-initrange, initrange)
            if layer in lin_layers:
                layer.bias.data.fill_(0)
                

### 3) Evaluation Metric

In [131]:
from sklearn.metrics import precision_score,recall_score,average_precision_score

def round_manual(data, threshold):
    return (data >= threshold).astype(int)

def calculate_accuracy(predicted, actuals, num_labels):
    """
    @param predicted: data type = Variable
    @param actuals: data type = Variable
    @param num_labels: no of go terms
    @return: accuracy measure
    """
    predicted = np.round(predicted.data.numpy())
    total_predictions = actuals.size()[0]
    accuracy = np.sum(predicted==actuals.data.numpy())/(total_predictions*num_labels)
    return accuracy

def m_tau(predictions):
    return len([np.sum(i) for i in predictions if np.sum(i)!=0])

def n_e(predictions):
    return predictions.shape[0]



def calculate_recall_precision(predicted, actual):
    '''
    Overall, this function calculates the recall and precision of the validation set proteins.
    The function FIRST calculates the precision and recall values of INDIVIDUAL proteins. 
    It then takes the mean average of these values to get "dataset-level" precision and recall.
    '''
    
    PositivesPerRow = actual.numpy().sum(axis=1) #number of functions for each protein
    PosPredictionsPerRow = predicted.sum(axis=1) #number of predictions for each protein
    TPs = np.multiply(actual.numpy(), predicted) #element-wise multiplication: 1 if TP, else 0
    TPsPerRow = TPs.sum(axis=1) #number of true positives for each protein
    
    #PrecisionPerRow (Protein) - if protein has 0 positive predictions, the protein's precision = 0.
    #Else, the protein's precision = TPs/PositivePreds
    PrecisionPerRow = np.where(PosPredictionsPerRow == 0, 0, TPsPerRow/PosPredictionsPerRow)
    RecallPerRow = np.where(PositivesPerRow==0, 0, TPsPerRow/PositivesPerRow) #Recall per Protein
    
    #RecallScore = average of individual protein recall scores
    RecallScore = sum(RecallPerRow)/len(RecallPerRow) #denominator is non-zero
    
    #PrecisionScore = average of CERTAIN individual protein precision scores (see line below)
    #Only consider rows with at least one predicted Go-Term.
    #Note that some proteins can have Precision=0 but still have predictions.
    if sum(PrecisionPerRow)>0:
        PrecisionScore = sum(PrecisionPerRow)/len([x for x in PosPredictionsPerRow if x!=0]) 
    else:
        PrecisionScore = 0
    return RecallScore, PrecisionScore
    
    
def F_score(predicted, actuals):
    """
    @param predicted: data type = Variable
    @param actuals: data type = Variable
    @return: Maximum f score over all values of tau and the corresponding tau threshold
    """
    f_max, optimal_threshold, optimal_precision, optimal_recall = 0, 0, 0, 0
    for threshold in [i/100 for i in range(1,100)]:
        predicted_tau = round_manual(predicted.data.numpy(), threshold)
        recall_score, precision_score = calculate_recall_precision(predicted_tau, actuals)
        
        if recall_score==0 and precision_score==0:
            output = 0
        else:
            output = ((2*precision_score*recall_score) / (precision_score + recall_score))
        if output > f_max:
            f_max = output
            optimal_threshold = threshold
            optimal_precision = precision_score
            optimal_recall = recall_score
    
    return f_max, optimal_threshold, optimal_precision, optimal_recall

### 4) Early stop condition and training stage

In [132]:
def early_stop(val_acc_history, t=10, required_progress=0.001):
    """
    Stop the training if there is no non-trivial progress in k steps
    @param val_acc_history: a list contains all the historical validation acc
    @param required_progress: the next acc should be higher than the previous by 
        at least required_progress amount to be non-trivial
    @param t: number of training steps 
    @return: a boolean indicates if the model should earily stop
    """
    # TODO: add your code here
    
    cnt = 0 # initialize the count --> to store count of cases where difference in
                                    #  accuracy is less than required progress.
    
    if(len(val_acc_history) > 0): # if list has size > 0 
        for i in range(t): # start the loop
            index = len(val_acc_history) - (i+1) # start from the last term in list and move to the left
            if (index >= 1): # to check if index != 0 --> else we can't compare to previous value
                if ((val_acc_history[index] - val_acc_history[index-1]) < required_progress):
                    cnt += 1 # increase the count value
                else:
                    break # break if difference is greater 
    
    if(cnt != t): # if count is equal to t, return True
        return False
    else:
        return True

    
def train_test(valid_sequences, valid_label, valid_length, num_epochs, optimizer, data_iter, model, training_length, lstm=False):
    losses = []
    total_batches = int(training_length/ batch_size)
    validation_acc_history = []
    
    for epoch in range(1, num_epochs+1):
        stop_training = False
        for i, (train_data, train_labels, length_batch) in enumerate(data_iter):
            model.train()
            model.zero_grad()
            
            if lstm:
                hidden, c_t = model.init_hidden()
                outputs, hidden = model(train_data, hidden, c_t)
            else:
                outputs = model(train_data, length_batch)

            loss = criterion(outputs, train_labels.float())
            losses.append(loss.data[0])
            loss.backward()
            optimizer.step()

            model.eval()
            if lstm:
                hidden, c_t = model.init_hidden()
                print(hidden.size())
                print(valid_sequences.size())
                val_outputs, hidden = model(Variable(valid_sequences), hidden, c_t)
            else:
                val_outputs = model(Variable(valid_sequences), Variable(valid_length))
            
            val_accuracy = calculate_accuracy(val_outputs, Variable(valid_label), num_labels)
            validation_acc_history.append(val_accuracy)
            f_score,threshold,precision,recall = F_score(val_outputs, valid_label)
            stop_training = early_stop(validation_acc_history)

            if stop_training:
                print("earily stop triggered")
                break
            
            if (i+1) % batch_size == 0:
                print('Epoch: [{}/{}], Step: [{}/{}], Train loss: {}, \nF_Score: {}, Threshold: {}, Precision: {}, Recall: {}, Validation Acc: {}'\
                      .format(epoch, num_epochs, i+1, total_batches, np.mean(losses)/(total_batches*epoch), \
                        f_score, threshold, precision,recall, val_accuracy))
        
        if stop_training == True:
            break

### 5) Prediction Test Data performance

In [119]:
def accuracy_on_test_set(model, test_input_seq, test_seq_length, test_output_labels, num_labels):
    test_input_seq = Variable(test_input_seq)
    test_seq_length = Variable(test_seq_length)
    test_output_labels = Variable(test_output_labels)
    predicted = model(test_input_seq, test_seq_length)
    accuracy_on_test_set = calculate_accuracy(predicted, test_output_labels, num_labels)
    return accuracy_on_test_set

def FScore_on_test_set(model, test_input_seq, test_seq_length, test_output_labels, num_labels):
    test_input_seq = Variable(test_input_seq)
    test_seq_length = Variable(test_seq_length)
    #test_output_labels = Variable(test_output_labels)
    predicted = model(test_input_seq, test_seq_length)
    fmax, optimal_threshold, optimal_precision, optimal_recall = F_score(predicted, test_output_labels)
    return fmax, optimal_threshold, optimal_precision, optimal_recall

## 6) Model training and test performance - FastText

#### Hyperparameters 

In [55]:
learning_rate = 0.001
vocab_size = 26 # number words in the vocabulary base
emb_dim = 20 # dimension for n-gram embedding
num_epochs = 500 # number epoch to train
batch_size = 20

### PART I - Human Data:

In [60]:
data_size = len(human_train_sequences)
num_labels = human_GO_terms.shape[0] #147

model = FastText(vocab_size, emb_dim, num_labels)
criterion = nn.MultiLabelSoftMarginLoss()  
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) 

In [61]:
data_iter = batch_iter(human_train_vectors, human_train_labels, human_train_lengths, batch_size)

# Model Training
train_test(human_valid_vectors, human_valid_labels, human_valid_lengths, \
           num_epochs, optimizer, data_iter, model, data_size) 

FScore,_,Precision,Recall = FScore_on_test_set(model, human_test_vectors, human_test_lengths,human_test_labels, num_labels)

# Prediction on test set
print("Test Data F-Score for yeast protein prediction is", FScore, '\nPrecision:',Precision,
      '\nRecall:',Recall)

/Users/Brenton/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:41: RuntimeWarning: invalid value encountered in true_divide
/Users/Brenton/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:40: RuntimeWarning: invalid value encountered in true_divide


Epoch: [1/500], Step: [20/251], Train loss: 0.0035386310274382513, 
F_Score: 0.08519872708787733, Threshold: 0.21, Precision: 0.04470668031957694, Recall: 0.9037445863825889, Validation Acc: 0.8863008908032343
Epoch: [1/500], Step: [40/251], Train loss: 0.003298604547739979, 
F_Score: 0.08529061347965258, Threshold: 0.08, Precision: 0.04470155305494888, Recall: 0.9270832135989816, Validation Acc: 0.9508397520723608
earily stop triggered
Test Data F-Score for yeast protein prediction is 0.0719887130174 
Precision: 0.0395926571975 
Recall: 0.396051453599


### PART II - Yeast Data:

In [62]:
data_size = len(yeast_train_sequences) #3447
num_labels = yeast_GO_terms.shape[0] #26

model = FastText(vocab_size, emb_dim, num_labels)
data_iter = batch_iter(yeast_train_vectors, yeast_train_labels, yeast_train_lengths, batch_size)

# Model Training
train_test(yeast_valid_vectors, yeast_valid_labels, yeast_valid_lengths, \
           num_epochs, optimizer, data_iter, model, data_size)

FScore,_,Precision,Recall = FScore_on_test_set(model, yeast_test_vectors, yeast_test_lengths,yeast_test_labels, num_labels)

# Prediction on test set
print("Test Data F-Score for yeast protein prediction is", FScore, '\nPrecision:',Precision,
      '\nRecall:',Recall)

/Users/Brenton/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:41: RuntimeWarning: invalid value encountered in true_divide
/Users/Brenton/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:40: RuntimeWarning: invalid value encountered in true_divide


earily stop triggered
Test Data F-Score for yeast protein prediction is 0.160890942372 
Precision: 0.0906665987734 
Recall: 0.71359223301


## 6B) Model training and test performance - LSTM

#### Part I - Human Data 

In [133]:
learning_rate = 0.001
vocab_size = 26 # number words in the vocabulary base
emb_dim = 20 # dimension for n-gram embedding
num_epochs = 500 # number epoch to train
batch_size = 40

#New parameters
hidden_size=20

In [ ]:
data_size = len(human_train_sequences)
num_labels = human_GO_terms.shape[0] #147

model = LSTM(vocab_size, emb_dim, hidden_size, num_labels, batch_size)
criterion = nn.MultiLabelSoftMarginLoss()  
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) 

data_iter = batch_iter(human_train_vectors, human_train_labels, human_train_lengths, batch_size)

# Model Training
train_test(human_valid_vectors, human_valid_labels, human_valid_lengths, \
           num_epochs, optimizer, data_iter, model, data_size, lstm=True) 

FScore,_,Precision,Recall = FScore_on_test_set(model, human_test_vectors, human_test_lengths,human_test_labels, num_labels)

# Prediction on test set
print("Test Data F-Score for human protein prediction is", FScore, '\nPrecision:',Precision,
      '\nRecall:',Recall)

In [120]:
data_size = len(yeast_train_sequences) #3447
num_labels = yeast_GO_terms.shape[0] #26

model = LSTM(vocab_size, emb_dim, hidden_size, num_labels, batch_size)
data_iter = batch_iter(yeast_train_vectors, yeast_train_labels, yeast_train_lengths, batch_size)

# Model Training
train_test(yeast_valid_vectors, yeast_valid_labels, yeast_valid_lengths, \
           num_epochs, optimizer, data_iter, model, data_size, lstm=True)

FScore,_,Precision,Recall = FScore_on_test_set(model, yeast_test_vectors, yeast_test_lengths,yeast_test_labels, num_labels)

# Prediction on test set
print("Test Data F-Score for yeast protein prediction is", FScore, '\nPrecision:',Precision,
      '\nRecall:',Recall)

torch.Size([40, 20])
torch.Size([963, 4092])


RuntimeError: inconsistent tensor sizes at /Users/soumith/code/builder/wheel/pytorch-src/torch/lib/TH/generic/THTensorMath.c:2709